This example shows how to solve the [FAST quick start example](https://web.stanford.edu/~lcambier/fast/demo.php) using this package.
We start by setting the different constants:

In [1]:
const num_stages = 2
const numScen = 2
const C = 1
const P = 2
const d = [2, 3];

We now model the master problem using [StructJuMP](https://github.com/StructJuMP/StructJuMP.jl).

In [2]:
using StructJuMP
m1 = StructuredModel(num_scenarios=numScen)
@variable(m1, x >= 0)
@objective(m1, Min, C * x);

INFO: Recompiling stale cache file /home/blegat/.julia/lib/v0.6/JuMP.ji for module JuMP.
INFO: Recompiling stale cache file /home/blegat/.julia/lib/v0.6/StructJuMP.ji for module StructJuMP.


For each of the two scenarios we need to create a [StructJuMP](https://github.com/StructJuMP/StructJuMP.jl) model specifying that `m1` is the parent and that the scenario has probability `1/2`.

In [3]:
for ξ in 1:numScen
    m2 = StructuredModel(parent=m1, prob=1/2, id=ξ)
    @variable(m2, s >= 0)
    @constraints m2 begin
        s <= d[ξ]
        s <= x
    end
    @objective(m2, Max, P * s)
end

We first need to pick an LP solver, see [here](http://www.juliaopt.org/) for a list of the available choices.

In [4]:
using GLPKMathProgInterface
solver = GLPKMathProgInterface.GLPKSolverLP();

This structured model need to be transformed into an appropriate structure to run SDDP on it.
This is achieved by `stochasticprogram`:

In [5]:
using CutPruners
const pruner = AvgCutPruningAlgo(-1)
using StructDualDynProg
const SOI = StructDualDynProg.StochOptInterface
sp = SOI.stochasticprogram(m1, num_stages, solver, pruner);

INFO: Recompiling stale cache file /home/blegat/.julia/lib/v0.6/StructDualDynProg.ji for module StructDualDynProg.


We can now run the sddp algorithm on it using `SDDP`. We are using 2 forward paths per iteration and we stop either after 10 iterations or once the pereira criterion is satisfied with $\alpha = 0.1$.

In [6]:
algo = StructDualDynProg.SDDP.Algorithm(K = 2)
sol = SOI.optimize!(sp, algo, SOI.Pereira(0.1) | SOI.IterLimit(10))

Lower Bound: -2.0
Upper Bound: -2.0

 ──────────────────────────────────────────────────────────────────────
                               Time                   Allocations      
                       ──────────────────────   ───────────────────────
   Tot / % measured:        3.69s / 81.8%            102MiB / 83.1%    

 Section       ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────
 

We can verify that the algorithm have found the right value by inspecting the solution:

In [7]:
sol.objval

LoadError: [91mtype Info has no field objval[39m

iteration 1        1    2.78s  92.1%   2.78s   82.5MiB  97.0%  82.5MiB
   solve            3    1.59s  52.8%   530ms   37.4MiB  44.0%  12.5MiB
   ocuts            2    101ms  3.35%  50.6ms   3.99MiB  4.69%  2.00MiB
   setx             2   9.34ms  0.31%  4.67ms    178KiB  0.20%  89.2KiB
   merged           1   1.55μs  0.00%  1.55μs         -  0.00%        -
 iteration 2        1    176ms  5.83%   176ms    936KiB  1.07%   936KiB
   solve            3    176ms  5.82%  58.5ms    892KiB  1.02%   297KiB
   ocuts            2   28.6μs  0.00%  14.3μs   3.47KiB  0.00%  1.73KiB
   setx             2   1.10μs  0.00%   551ns         -  0.00%        -
   merged           1    825ns  0.00%   825ns         -  0.00%        -
 iteration 3        1   38.0ms  1.26%  38.0ms   1.05MiB  1.24%  1.05MiB
   setx             2   9.76ms  0.32%  4.88ms    173KiB  0.20%  86.4KiB
   solve            3    434μs  0.01%   145μs   74.4KiB  0.09%  24.8KiB
   ocuts            1   36.4μs  0.00%  36.4μs   1.72KiB  0.00%  1